# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 14 2023 2:06PM,259302,10,EYE43032,Eye Pharma Inc,Released
1,Mar 14 2023 2:02PM,259301,10,U1241,"Uniderm USA, Inc",Released
2,Mar 14 2023 2:00PM,259300,10,9265780,"CLINUVEL, Inc.",Released
3,Mar 14 2023 1:50PM,259298,19,012658,"Emersa Waterbox, LLC",Released
4,Mar 14 2023 1:38PM,259297,21,RHL031423,"NBTY Global, Inc.",Released
5,Mar 14 2023 1:34PM,259296,19,012657,"Emersa Waterbox, LLC",Released
6,Mar 14 2023 1:20PM,259294,19,012656,"Emersa Waterbox, LLC",Released
7,Mar 14 2023 1:17PM,259292,19,ADV80013718,"AdvaGen Pharma, Ltd",Released
8,Mar 14 2023 1:17PM,259292,19,ADV80013719,"AdvaGen Pharma, Ltd",Released
9,Mar 14 2023 1:17PM,259292,19,ADV80013720,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,259297,Released,1
34,259298,Released,1
35,259300,Released,1
36,259301,Released,1
37,259302,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259297,NaN,NaN,1.0
259298,NaN,NaN,1.0
259300,NaN,NaN,1.0
259301,NaN,NaN,1.0
259302,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259195,46.0,10.0,8.0
259215,0.0,6.0,4.0
259225,0.0,0.0,1.0
259243,15.0,1.0,0.0
259258,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259195,46,10,8
259215,0,6,4
259225,0,0,1
259243,15,1,0
259258,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259195,46,10,8
1,259215,0,6,4
2,259225,0,0,1
3,259243,15,1,0
4,259258,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259195,46,10,8
1,259215,,6,4
2,259225,,,1
3,259243,15,1,
4,259258,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc"
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc."
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC"
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc."
5,Mar 14 2023 1:34PM,259296,19,"Emersa Waterbox, LLC"
6,Mar 14 2023 1:20PM,259294,19,"Emersa Waterbox, LLC"
7,Mar 14 2023 1:17PM,259292,19,"AdvaGen Pharma, Ltd"
13,Mar 14 2023 1:17PM,259293,10,"Snap Medical Industries, LLC"
14,Mar 14 2023 1:10PM,259290,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc,,,1
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc",,,1
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc.",,,1
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC",,,1
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc.",,,1
5,Mar 14 2023 1:34PM,259296,19,"Emersa Waterbox, LLC",,,1
6,Mar 14 2023 1:20PM,259294,19,"Emersa Waterbox, LLC",,,1
7,Mar 14 2023 1:17PM,259292,19,"AdvaGen Pharma, Ltd",,1,5
8,Mar 14 2023 1:17PM,259293,10,"Snap Medical Industries, LLC",,,1
9,Mar 14 2023 1:10PM,259290,10,"Snap Medical Industries, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc,1,,
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc",1,,
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc.",1,,
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC",1,,
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc.",1,,
5,Mar 14 2023 1:34PM,259296,19,"Emersa Waterbox, LLC",1,,
6,Mar 14 2023 1:20PM,259294,19,"Emersa Waterbox, LLC",1,,
7,Mar 14 2023 1:17PM,259292,19,"AdvaGen Pharma, Ltd",5,1,
8,Mar 14 2023 1:17PM,259293,10,"Snap Medical Industries, LLC",1,,
9,Mar 14 2023 1:10PM,259290,10,"Snap Medical Industries, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc,1,,
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc",1,,
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc.",1,,
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC",1,,
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc,1.0,NaN,NaN
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc",1.0,NaN,NaN
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc.",1.0,NaN,NaN
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 14 2023 2:06PM,259302,10,Eye Pharma Inc,1.0,0.0,0.0
1,Mar 14 2023 2:02PM,259301,10,"Uniderm USA, Inc",1.0,0.0,0.0
2,Mar 14 2023 2:00PM,259300,10,"CLINUVEL, Inc.",1.0,0.0,0.0
3,Mar 14 2023 1:50PM,259298,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Mar 14 2023 1:38PM,259297,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4148497,42.0,36.0,26.0
15,518410,12.0,16.0,46.0
19,1555723,10.0,1.0,0.0
20,259225,1.0,0.0,0.0
21,518573,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4148497,42.0,36.0,26.0
1,15,518410,12.0,16.0,46.0
2,19,1555723,10.0,1.0,0.0
3,20,259225,1.0,0.0,0.0
4,21,518573,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,36.0,26.0
1,15,12.0,16.0,46.0
2,19,10.0,1.0,0.0
3,20,1.0,0.0,0.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,15,Released,12.0
2,19,Released,10.0
3,20,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,26.0,46.0,0.0,0.0,0.0
Executing,36.0,16.0,1.0,0.0,1.0
Released,42.0,12.0,10.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,26.0,46.0,0.0,0.0,0.0
1,Executing,36.0,16.0,1.0,0.0,1.0
2,Released,42.0,12.0,10.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,26.0,46.0,0.0,0.0,0.0
1,Executing,36.0,16.0,1.0,0.0,1.0
2,Released,42.0,12.0,10.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()